## First Layer

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1248, 768)

driver.get("https://ba-sh.com/us/clothing/")
time.sleep(5.5)
x_button = driver.find_element_by_xpath('/html/body/div[3]/div/button')
x_button.click()

counter = 1
for i in range(0,7):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollBy(0, 768);")

        # Wait to load page
        time.sleep(.3)

        # Calculate new scroll height and compare with last scroll height
        new_height = 768 * counter
        if new_height >= last_height:
            break

        counter += 1

    driver.execute_script("window.scrollBy(0, -768);")
    time.sleep(.3)
    load_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[5]/div[1]/button')
    load_button.click()
    time.sleep(1)

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollBy(0, 768);")

    # Wait to load page
    time.sleep(.3)

    # Calculate new scroll height and compare with last scroll height
    new_height = 768 * counter
    if new_height >= last_height:
        break

    counter += 1

soup = BeautifulSoup(driver.page_source,'html.parser')


In [26]:
product_name = []
original_price = []
sale_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []

for i in soup.find_all('div',{'class':'ProductTile-scrolltag'}):
    if i.find('a',{'class':'ProductTile-link js-ProductTile-link'}) != None:
        product_name.append(i.find('a',{'class':'ProductTile-link js-ProductTile-link'}).text.strip())
    else:
        product_name.append(None)

    if i.find('span',{'class':'Product-priceNew'}) != None:
        sale_price.append((i.find('span',{'class':'Product-priceNew'})).text.strip())
        if i.find('span',{'class':'Product-priceOld'}) != None:
            original_price.append((i.find('span',{'class':'Product-priceOld'})).text.strip())
        else:
            original_price.append(None)
    else:
        sale_price.append(None)
        original_price.append(None)

    product_url.append("https://ba-sh.com"+i.find('a',{'class':'ProductTile-link js-ProductTile-link'}).get('href'))

    if (i.find('img',{'class':'ProductTile-image js-ProductTile-image'})) !=None:
        image_url.append((i.find('img',{'class':'ProductTile-image js-ProductTile-image'})).get('src'))
    else:
        image_url.append(None)
    brand.append("ba&sh")
    product_type.append("Women's Clothing")
    timestamp.append(date_today)

In [32]:
cleaned_name = []
for name in product_name:
    cleaned_name.append(name.replace(".\n\n\n"," "))

In [33]:
column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = cleaned_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [34]:
df

,product_name,original_price,sale_price,product_link,image_link
0,SOMA DRESS,None,$295.00,https://ba-sh.com/us/p/dress-soma-noir-3664784...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
1,JUNE Wool sweater,None,$325.00,https://ba-sh.com/us/p/sweater-june-noir-36647...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
2,ALINE Print midi dress,$420.00,$210.00,https://ba-sh.com/us/p/dress-aline-noir-366478...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
3,KAWAI BLOUSE,None,$280.00,https://ba-sh.com/us/p/blouse-kawai-ecru-36647...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
4,SHAYA DRESS,None,$365.00,https://ba-sh.com/us/p/dress-shaya-noir-366478...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
...,...,...,...,...,...
452,WINNIE Sweater dress,$320.00,$160.00,https://ba-sh.com/us/p/dress-winnie-noir-36647...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
453,SECRET SWEATER,$195.00,$117.00,https://ba-sh.com/us/p/sweater-secret-noir-366...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
454,SALLY JEAN,None,$175.00,https://ba-sh.com/us/p/jean-sally-lightusedblu...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...
455,SALLY Boyfriend Jeans,None,$250.00,https://ba-sh.com/us/p/jeans-sally-worndark-36...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...


## Second Layer

In [42]:
product_description = []
material = []
picture_name = []
color = []
counter = 1
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')

for link in product_url:
    driver.get(link)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    if soup.find('span',{'class':'Product-name'}) != None:

        item_color = []
        if soup.find('ul',{'class':'Product-variationsList'}).find('a',{'class':"Product-colorsLink Product-colorsLink--selected"}) != None:
            for color_i in soup.find('ul',{'class':'Product-variationsList'}).find_all('a'):
                item_color.append(color_i.get("title"))
            color.append(item_color)
        else:
            color.append(None)

        if soup.find('p',{'class':'ProductInfos-contentText'}) != None:
            product_description.append(soup.find('p',{'class':'ProductInfos-contentText'}).text.strip())
        else:
            product_description.append(None)

        material_item = []
        if soup.find('div',{'class':'ProductInfos-content'}) != None:
            for j in soup.find_all('div',{'class':'ProductInfos-content'}):
                for i in j.find_all('li',{'class':'ProductInfos-listItem'}):
                    material_item.append(i.text.strip())
            material.append(material_item)
        else:
            material.append(None)
    else:
        color.append(None)
        product_description.append(None)
        material.append(None)

    picture_name.append('ba&sh_' + str(counter) + '.jpg') # change variable
    counter += 1
driver.close()

In [45]:
color

[['Select colorERP: BLACK', 'Select colorERP: ORANGE'],
 ['Select colorERP: GREY', 'Select colorERP: BLACK'],
 ['Select colorERP: BLACK', 'Select colorERP: ORANGE'],
 ['Select colorERP: OFF WHITE'],
 ['Select colorERP: BLACK', 'Select colorERP: ORANGE'],
 ['Select colorERP: OFF WHITE', 'Select colorERP: GREEN'],
 ['Select colorERP: BLACK', 'Select colorERP: ORANGE'],
 ['Select colorERP: BLACK'],
 ['Select colorERP: BLACK'],
 ['Select colorERP: PURPLE'],
 ['Select colorERP: BLACK', 'Select colorERP: BEIGE'],
 ['Select colorERP: GREY', 'Select colorERP: BLACK'],
 ['Select colorERP: BLACK'],
 ['Select colorERP: BLACK'],
 ['Select colorERP: BLUE'],
 ['Select colorERP: OFF WHITE'],
 ['Select colorERP: RED', 'Select colorERP: BLACK'],
 ['Select colorERP: OFF WHITE', 'Select colorERP: BLACK'],
 ['Select colorERP: OFF WHITE', 'Select colorERP: BLACK'],
 ['Select colorERP: BLACK', 'Select colorERP: ORANGE'],
 ['Select colorERP: OFF WHITE', 'Select colorERP: RED'],
 ['Select colorERP: OFF WHITE'

In [44]:
 product_description

["A ba&sh dress with everything you love about ba&sh dresses! Fluid, stylish and easy to wear. This SOMA dress has lots going for it and you'll love the short floral style, V-neckline and button placket down the front. The waist is cinched with a piped panel with gathers. A feminine and joyful dress that will brighten up your day. - Short floral dress- V-neckline- Short sleeves- 100% viscose",
 "This ba&sh JUNE cardigan is made in a warm and soft knit with sophisticated details. The cardigan has a V-neck and ¾-length sleeves. It stands apart with its original bejeweled buttons which add a touch of refinement. A cardigan which is both warm and elegant.  - Women's cardigan- Novel buttons- 53% wool, 30% polyamide, 17% alpaca",
 "You'll fall for this ba&sh ALINE dress and the shiny print which will light up your winter closet. This midi dress is cut in a flowing fabric and it has lots of feminine details: A V-neckline extending to a button placket shirt-dress style, ¾-length sleeves with s

In [49]:
counter = 1
picture_name =[]
for i in range(0,457):
    picture_name.append('ba&sh_' + str(counter) + '.jpg') # change variable
    counter += 1

In [53]:
# data cleaning for material
material_clean =[]
for i,v in enumerate(material):
    if v != None:
        item_material = []
        for j in v:
            if "%" in j:
                item_material.append(j.replace("Material:\n",""))
        material_clean.append(item_material)
    else:
        material_clean.append(None)

In [54]:
material_clean

[['100%VI'],
 ['53%WO 30%PA 17%WP'],
 ['100%VI'],
 ['100%CO'],
 ['100%VI'],
 ['69%WP 29%PA 2%EA'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%PL'],
 ['100%PL'],
 ['53%WO 30%PA 17%WP'],
 ['52%MD 48%CO'],
 ['100%VI'],
 ['99%CO 1%EA'],
 ['97%CO 3%EA'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['51%CO 47%LI 2%EA'],
 ['100%PL'],
 ['69%SE 31%FM'],
 ['100%VI'],
 ['100%VI'],
 ['100%PL'],
 ['100%VI'],
 ['98%CO 2%EA'],
 ['98%CO 2%EA'],
 ['100%VI'],
 ['100%PL'],
 ['100%CO'],
 ['100%PL'],
 ['98%VI 2%FM'],
 ['71%CLY 20%LI 9%PL'],
 ['100%VI'],
 ['50%PL 50%CO'],
 ['100%VI'],
 ['100%WS'],
 ['100%VI'],
 ['100%VI'],
 ['100%VI'],
 ['73%WO 21%PA 4%PC 2%VI'],
 ['98%CO 2%EA'],
 ['100%PL'],
 ['100%VI'],
 ['85%VI 15%FM'],
 ['100%CAG'],
 ['98%VI 2%FM'],
 ['96%CO 4%EA'],
 ['100%VI'],
 ['68%SE 32%FM'],
 ['100%VI'],
 ['37%WO 34%WS 29%CO'],
 ['85%VI 15%FM'],
 ['100%CO'],
 ['100%VI'],
 ['100%CO'],
 ['85%VI 15%FM'],
 ['100%V

In [55]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = cleaned_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material_clean
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,ba&sh,Women's Clothing,SOMA DRESS,A ba&sh dress with everything you love about b...,None,$295.00,"[Select colorERP: BLACK, Select colorERP: ORANGE]",[100%VI],https://ba-sh.com/us/p/dress-soma-noir-3664784...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_1.jpg
1,ba&sh,Women's Clothing,JUNE Wool sweater,This ba&sh JUNE cardigan is made in a warm and...,None,$325.00,"[Select colorERP: GREY, Select colorERP: BLACK]",[53%WO 30%PA 17%WP],https://ba-sh.com/us/p/sweater-june-noir-36647...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_2.jpg
2,ba&sh,Women's Clothing,ALINE Print midi dress,You'll fall for this ba&sh ALINE dress and the...,$420.00,$210.00,"[Select colorERP: BLACK, Select colorERP: ORANGE]",[100%VI],https://ba-sh.com/us/p/dress-aline-noir-366478...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_3.jpg
3,ba&sh,Women's Clothing,KAWAI BLOUSE,This seductive ba&sh KAWA blouse with unique c...,None,$280.00,[Select colorERP: OFF WHITE],[100%CO],https://ba-sh.com/us/p/blouse-kawai-ecru-36647...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_4.jpg
4,ba&sh,Women's Clothing,SHAYA DRESS,This ba&sh SHAYA dress holds all the cards whe...,None,$365.00,"[Select colorERP: BLACK, Select colorERP: ORANGE]",[100%VI],https://ba-sh.com/us/p/dress-shaya-noir-366478...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
452,ba&sh,Women's Clothing,WINNIE Sweater dress,"This ba&sh WINNIE dress is alluring, trendy an...",$320.00,$160.00,"[Select colorERP: BLACK, Select colorERP: GREY]",[80%WO 20%PA],https://ba-sh.com/us/p/dress-winnie-noir-36647...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_453.jpg
453,ba&sh,Women's Clothing,SECRET SWEATER,This ba&sh SECRET turtleneck sweater drips wit...,$195.00,$117.00,"[Select colorERP: BLACK, Select colorERP: GREY]",[100%WME],https://ba-sh.com/us/p/sweater-secret-noir-366...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_454.jpg
454,ba&sh,Women's Clothing,SALLY JEAN,These ba&sh jeans strike a perfect harmony bet...,None,$175.00,[Select colorERP: BLUE],[98%CO 2%EA],https://ba-sh.com/us/p/jean-sally-lightusedblu...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_455.jpg
455,ba&sh,Women's Clothing,SALLY Boyfriend Jeans,,None,$250.00,[Select colorERP: BLUE],[98%CO 2%EA],https://ba-sh.com/us/p/jeans-sally-worndark-36...,https://ba-sh.com/dw/image/v2/BBTP_PRD/on/dema...,2021-01-13,ba&sh_456.jpg


In [56]:
second_pass.to_csv("ba&sh.csv", index=False, encoding = 'utf-8-sig')

In [57]:
import os
import requests as req

new_pd = pd.read_csv('ba&sh.csv', encoding='utf-8-sig')

os.mkdir('ba&sh')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

    img_data = req.get(link,headers=headers).content
    with open('ba&sh/'+ "ba&sh_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1